### Day 12: Garden Groups

Link: https://adventofcode.com/2024/day/12#part2

To solve part two, we have to adjust the perimeter logic from part one. We can do that by enumerating each possible horizontal fence line from zero to the number of rows, and each possible vertical fence line from zero to the number of columns. A "fence" is then defined by its fence line and the position, or cell, it's added to. The goal is to identify all fences we need to add around a region and then check for their continuity to get the number of sides. For example, consider the arrangement below:

```
AABAA
BBBBA
AABAA
```

In this example, `B` uses four horizontal fence lines: 0, 1, 2, and 3; and four vertical fence lines: 0, 2, 3, and 4. Horizontal fence line #1 spans columns 0, 1, and 3. Note there's a break in its continuity, so that means that fence line has two sides. Similarly, vertical fence line #2 spans rows 0 and 2, also adding two sides.

When visiting a position, we assume four new fences, each stored in a separate variable determining its direction: top, bottom, left, or right. Similarly to the previous perimeter logic, we remove fences as we move to an adjacent position of the same type, but this time taking the direction we're moving into account. After completing a region, we check fence line continuity on each of the four possible fence directions to count the number of sides. Note that we must treat top and bottom, and left and right fences as separate, even if they use the same horizontal or vertical fence lines. That is because, as stated by the problem, fences might touch diagonally but should not count as if they crossed each other. Treating all horizontal or vertical fence lines as the same would lead to mistakenly assuming continuity in some cases.

In [ ]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
garden: list[list[str]] = []


for line in lines:
    row = list(line.strip())
    garden.append(row)


garden_positions = set([
    (row, column) for row in range(len(garden)) for column in range(len(garden[row]))
])
total_cost = 0
actions = [
    (-1, 0),  # Up
    (1, 0),  # Down
    (0, -1),  # Left
    (0, 1),  # Right
]


def is_within_garden(row: int, column: int) -> bool:
    return 0 <= row < len(garden) and 0 <= column < len(garden[0])


def merge_fence_dictionaries(
    current_fence_dictionary: dict[int, set[int]],
    new_fence_dictionary: dict[int, set[int]],
) -> None:
    for fence, positions in new_fence_dictionary.items():
        current_fence_dictionary.setdefault(fence, set())
        current_fence_dictionary[fence] |= positions


def count_continuous_fences(fence_dictionary: dict[int, set[int]]) -> int:
    count = 0

    for positions in fence_dictionary.values():
        sorted_positions = sorted(positions)
        count += 1

        for idx in range(1, len(sorted_positions)):
            if sorted_positions[idx] - sorted_positions[idx - 1] > 1:  # Broke continuity
                count += 1

    return count


while garden_positions:
    row, column = garden_positions.pop()
    garden_positions.add((row, column))  # Reinsert position as it's expected to exist in the loop below
    to_visit = [(row, column)]
    area = 0
    unique_top_fences: dict[int, set[int]] = {}
    unique_bottom_fences: dict[int, set[int]] = {}
    unique_left_fences: dict[int, set[int]] = {}
    unique_right_fences: dict[int, set[int]] = {}

    while to_visit:
        row, column = to_visit.pop()

        if (row, column) not in garden_positions:
            continue

        garden_positions.remove((row, column))
        area += 1
        new_unique_top_fences = {row: {column}}
        new_unique_bottom_fences = {row + 1: {column}}
        new_unique_left_fences = {column: {row}}
        new_unique_right_fences = {column + 1: {row}}

        for add_row, add_column in actions:
            new_row, new_column = row + add_row, column + add_column

            if not is_within_garden(new_row, new_column):
                continue

            if garden[row][column] != garden[new_row][new_column]:
                continue

            to_visit.append((new_row, new_column))

            if add_row < 0:  # Up. Remove top fence
                new_unique_top_fences.pop(row)
            elif add_row > 0:  # Down. Remove bottom fence
                new_unique_bottom_fences.pop(row + 1)
            elif add_column < 0:  # Left. Remove left fence
                new_unique_left_fences.pop(column)
            elif add_column > 0:  # Right. Remove right fence
                new_unique_right_fences.pop(column + 1)

        merge_fence_dictionaries(unique_top_fences, new_unique_top_fences)
        merge_fence_dictionaries(unique_bottom_fences, new_unique_bottom_fences)
        merge_fence_dictionaries(unique_left_fences, new_unique_left_fences)
        merge_fence_dictionaries(unique_right_fences, new_unique_right_fences)

    perimeter = count_continuous_fences(unique_top_fences)
    perimeter += count_continuous_fences(unique_bottom_fences)
    perimeter += count_continuous_fences(unique_left_fences)
    perimeter += count_continuous_fences(unique_right_fences)
    total_cost += perimeter * area


print(total_cost)